## 📂 **Mounting Your Google Drive**

Before we run the notebook, we first need to "mount" Google Drive. This enables your Drive files to be accessible inside Colab.  

**Why do this?**

👉 You will save and read files (HAR files, outputs) directly from your Google Drive. There is no need to upload/download manually to and from Colab. Also, when using Colab anything that is kept there for the lifetime of the runtime will be lost when the runtime shuts down. Using Google Drive to handle files is so much more reliable.  

### **How to do it:**

1️⃣ Go to the code window below this section. It looks like this:  
<img src="https://raw.githubusercontent.com/rilhia/linkedin_content_analysis/main/images/RunCell_1.png?raw=true" width="500">

2️⃣ Click on the **play** ▶️ button:  
<img src="https://raw.githubusercontent.com/rilhia/linkedin_content_analysis/main/images/RunCell_2.png?raw=true" width="500">

3️⃣ At this point, you'll be guided through the process by Google. Depending on the configuration of your system, you will see different options. Once Google Drive is mounted, you will see this:  
<img src="https://raw.githubusercontent.com/rilhia/linkedin_content_analysis/main/images/MountedDrive.png?raw=true" width="300">

  

---

Once your Drive is mounted and libraries installed, you're ready to start! How do you start?





In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 📦 **Installing required libraries**

Colab provides many useful libraries pre-installed — but sometimes we need to install extra ones.

In this notebook, we use **`!pip install`** to ensure we have the correct version of **`XlsxWriter`**, which we use to generate **`.xlsx`** Excel files.  
This step is safe to run and ensures compatibility when writing out the results.

---



In [ ]:
# Install XlsxWriter to enable Excel file production
!pip install XlsxWriter

## 📦 **Imports & Utility Functions**

Before we start working with the LinkedIn HAR data, we need to load some essential **Python libraries** and define a few **helper functions**.

### 🛠️ **What this section does:**

✅ Loads core libraries:

- `json` — to parse the HAR file (which is in JSON format)  
- `pandas` — for powerful data handling and analysis  
- `re` — for regular expressions (pattern matching, used to extract IDs, hashtags, etc.)  
- `numpy` — for numerical operations (used in interpolations)  
- `copy` — to safely duplicate data structures  
- `glob` — to easily load multiple HAR files  
- `datetime` — for converting and formatting timestamps
---


✅ Defines **utility functions**:

- 1️⃣ `safe_get()`  
Safely navigate a deeply nested dictionary without errors if something is missing. Very useful when dealing with LinkedIn’s complex JSON.

- 2️⃣ `extract_file_segment()`  
Extracts a unique part of the LinkedIn image URL — this contains an **epoch timestamp** (which we will later use to figure out when posts were published).

- 3️⃣ `extract_epoch_from_segment()`  
Takes the image URL segment and extracts the actual **epoch time** (milliseconds since 1970) so we can convert it to a readable date/time.

- 4️⃣ `extract_entity_id()`  
Extracts **LinkedIn post or activity IDs** from special LinkedIn URN strings (e.g. `"urn:li:activity:1234567890123"`).

---

### 💡 **Why do this first?**

The raw HAR files from LinkedIn are:

- **Very large**
- **Highly nested**
- **Messy**

These utilities give us **clean, reusable ways** to pull out exactly what we need (timestamps, post IDs, text content, etc.) so that the rest of the notebook can work smoothly.

---

👉 After this section, we will load the HAR files and start extracting the **actual post data**!

But first, we press **play** ▶️ to run this code...

In [ ]:
# === Imports ===

# Core modules
import json
import pandas as pd
import re
import numpy as np
import copy
import glob
from datetime import datetime

# === Utility Functions ===

def safe_get(data, path, default=""):
    """
    Safely navigate a nested dictionary using a list of keys.

    Args:
        data (dict): The dictionary to search.
        path (list): A list of keys representing the path.
        default (any): Default value to return if the path doesn't exist.

    Returns:
        The value found at the specified path or the default value.
    """
    for key in path:
        if isinstance(data, dict) and key in data:
            data = data[key]
        else:
            return default
    return default if data is None else data


def extract_file_segment(item):
    """
    Extract the fileIdentifyingUrlPathSegment from an image artifact in the item.
    This is used to retrieve an embedded timestamp (epoch) from the image URL.

    Args:
        item (dict): A dictionary representing a LinkedIn post item.

    Returns:
        str: The extracted file segment, or empty string if not found.
    """
    try:
        images = item.get("content", {}).get("imageComponent", {}).get("images", [])
        for img in images:
            for attr in img.get("attributes", []):
                vector = attr.get("detailData", {}).get("vectorImage", {})
                for artifact in vector.get("artifacts", []):
                    if "fileIdentifyingUrlPathSegment" in artifact:
                        return artifact["fileIdentifyingUrlPathSegment"]
    except Exception:
        pass
    return ""


def extract_epoch_from_segment(segment):
    """
    Extract the epoch timestamp from an image fileIdentifyingUrlPathSegment.
    Example segment format: "/path/to/image/1716789123456?params"

    Args:
        segment (str): The URL path segment.

    Returns:
        int or None: The extracted epoch timestamp, or None if not valid.
    """
    try:
        parts = segment.split("/")
        if len(parts) >= 4:
            timestamp_str = parts[3].split("?")[0]
            if timestamp_str.isdigit():
                return int(timestamp_str)
    except Exception:
        pass
    return None


def extract_entity_id(urn_str):
    """
    Extract the numeric entity ID from a LinkedIn URN string.
    Example URN: "urn:li:activity:1234567890123"

    Args:
        urn_str (str): The URN string.

    Returns:
        int or None: The numeric ID, or None if not found.
    """
    match = re.search(r'\d{6,}', urn_str or "")
    return int(match.group()) if match else None

## 📂 **Loading HAR Files (Raw LinkedIn Data)**

Now that we have our **helper functions** ready, it's time to load the actual data!

### 🗂️ **What is a HAR file?**

- HAR stands for **HTTP Archive**. It’s a special file format that records the network traffic between your browser (Chrome) and a website (LinkedIn in this case).
- When you scroll through LinkedIn posts with **Developer Tools open**, the browser is fetching the **raw post data** and other details — this gets recorded in the HAR file.

### 🚀 **What this section does:**

✅ Optionally shows **how to load a single HAR file** (example is commented out — useful if you want to test one file and have several available in your storage location).

<img src="https://github.com/rilhia/linkedin_content_analysis/blob/main/images/SingleHARFile.png?raw=true" width="500">

The green box shows the comment quotes **`"""`** being commented out using a hash **`#`**.

The red box shows the comment quotes being used to comment out the **`ALL HAR files`** section.

1. You define the filename here:
    ```python
    filename = "/content/drive/MyDrive/LinkedInData/www.linkedin.com.har"
    ```


✅ Automatically **loads ALL HAR files** from your selected folder:

<img src="https://github.com/rilhia/linkedin_content_analysis/blob/main/images/MultipleHARFile.png?raw=true" width="500">

The red box shows the comment quotes being used to comment out the **`OPTIONAL`** section.

The green box shows the comment quotes **`"""`** being commented out using a hash **`#`**.

1. You define the folder here:
    ```python
    har_folder = "/content/drive/MyDrive/LinkedInData"
    ```

2. It finds all `.har` files in that folder:
    ```python
    har_files = glob.glob(f"{har_folder}/*.har")
    ```

3. It loops through each HAR file and extracts the key part:
    ```python
    entries = har_data.get("log", {}).get("entries", [])
    ```

4. It combines **all entries** into one big list: `all_entries`

✅ Prepares a **combined HAR data structure** (`combined_har_data`) so it looks like a full HAR file — this allows our later functions (like the post extractor) to run on **all files at once**.

---

### 💡 **Why do we merge multiple HAR files?**

- When browsing LinkedIn, your HAR file will only capture the posts you scroll past during that session.
- If you're researching a creator with **hundreds of posts**, you’ll probably want to do **multiple scroll & save passes** (and save multiple HAR files).
- This section lets you merge them automatically into one combined dataset.

---

👉 Next, we’ll run the **post extraction** to pull out the actual post content and engagement data!

In [ ]:
# === Load HAR Files ===

# Initialize an empty list to hold all log entries
all_entries = []

# OPTIONAL: Single HAR file example (commented out)
#"""
# Example of loading a single HAR file
filename = "/content/drive/MyDrive/LinkedInData/www.linkedin.com.har"

with open(filename, 'r', encoding='utf-8') as f:
    har_data = json.load(f)

    # Extract the 'entries' from this HAR file
    entries = har_data.get("log", {}).get("entries", [])

    # Add these entries to our master list
    all_entries.extend(entries)
#"""


# === Load ALL HAR files from folder ===
"""

# Define the folder where HAR files are stored
har_folder = "/content/drive/MyDrive/LinkedInData"

# Find all .har files in the folder
har_files = glob.glob(f"{har_folder}/*.har")

# Print how many HAR files were found
print(f"✅ Found {len(har_files)} HAR files")

# Loop through each HAR file and load its contents
for filename in har_files:
    print(f"Loading: {filename}")
    with open(filename, 'r', encoding='utf-8') as f:
        har_data = json.load(f)

        # Extract the 'entries' from this HAR file
        entries = har_data.get("log", {}).get("entries", [])

        # Add these entries to our master list
        all_entries.extend(entries)

"""

# Final print: total number of entries combined from all files
print(f"✅ Total entries loaded: {len(all_entries)}")

# === Prepare combined HAR data structure ===
# Some functions expect HAR data in full "har_data" format.
# We can fake a combined one using our all_entries list:
combined_har_data = {"log": {"entries": all_entries}}

## 📝 **Extracting Posts and Social Engagement Data**

Now that we’ve **loaded all our HAR files** into memory, it’s time to extract the **useful content**:

✅ Actual **LinkedIn posts**  
✅ **Engagement metrics** (likes, comments, shares, reactions)

---

### **What this section does:**

We define a function:

```python
extract_posts_and_social(har_data)
```

It takes our combined HAR data and pulls out:

1️⃣ Post Data → Stored in posts list  
2️⃣ Social Data → Stored in social_data list  

---

### **How does it work?**

1️⃣ It loops through each HAR “entry” (which is an HTTP request/response):  

```python
for entry in har_data.get("log", {}).get("entries", []):
```

2️⃣ It filters only LinkedIn GraphQL requests:  

These requests contain the actual post data in the HAR:  

```python
if not url.startswith("https://www.linkedin.com/voyager/api/graphql?"):
    continue
```

3️⃣ For each GraphQL response:  

It looks inside:  

```python
response_json.get("included", [])
```

4️⃣ It extracts two types of content:  

**A) Posts**  
	•	If the item is:  
  ```python
  "$type" == "com.linkedin.voyager.dash.feed.SocialActivityCounts"
  ```

We extract:  
	•	Post text  
	•	Author name & profile  
	•	Post ID  
	•	If it’s a reshare (and link to original post)  
	•	Embedded image timestamp (used to infer post time!)  
	•	Share URL  
	•	Various IDs we’ll need for linking posts together later  

**B) Social Data**.  
	•	If the item is:  
  ```python
  "$type" == "com.linkedin.voyager.dash.feed.SocialActivityCounts"
  ```

We extract:  
	•	Total reactions  
	•	Likes  
	•	Comments  
	•	Shares  
	•	Detailed reaction types (Appreciation, Empathy, Entertainment, etc.)  

---
### **Why do we separate posts and social data?**

Because LinkedIn returns them in different API items:  
	•	The “Update” object contains the post itself  
	•	The “SocialActivityCounts” object contains the engagement metrics

---
### **What happens at the end?**

We run the extraction:  

```python
posts, social_data = extract_posts_and_social(combined_har_data)
```

And we print a summary:  
```
✅ Extracted X posts and Y social records
```
---
### **Summary**

At this point, we now have two key lists ready for further processing:  

📄 posts → All posts with text, author, timestamps, resharing info  
📊 social_data → All reactions, likes, comments, shares  

---

👉 In the next step, we’ll deduplicate posts (LinkedIn sometimes sends duplicate data in HAR files).




In [ ]:
# === Extract posts & social data ===

def extract_posts_and_social(har_data):
    """
    Extracts:
    - Post content and metadata from LinkedIn HAR data
    - Social engagement counts (likes, comments, shares, etc.)

    Returns:
    - posts: List of dictionaries (one per post)
    - social_data: List of dictionaries (one per social engagement record)
    """

    posts = []
    social_data = []

    # Loop over all HAR entries (HTTP requests/responses)
    for entry in har_data.get("log", {}).get("entries", []):

        # Only process GraphQL requests (where LinkedIn post data lives)
        url = entry.get("request", {}).get("url", "")
        if not url.startswith("https://www.linkedin.com/voyager/api/graphql?"):
            continue

        # Try to decode JSON response
        text_data = entry.get("response", {}).get("content", {}).get("text", "")
        try:
            response_json = json.loads(text_data)
        except json.JSONDecodeError:
            continue  # Skip invalid JSON

        # Loop through all "included" items in this GraphQL response
        for item in response_json.get("included", []):

            # === If this is a Post ===
            if item.get("$type") == "com.linkedin.voyager.dash.feed.Update":

                # Extract profile info (actor)
                attributes = safe_get(item, ["actor", "name", "attributesV2"], [])
                actor_profile = safe_get(attributes[0], ["detailData", "*profileFullName"]) if attributes else ""
                actor_profile_id = actor_profile.split("profile:")[1] if "profile:" in actor_profile else ""

                # Extract image timestamp (used for post time)
                file_segment = extract_file_segment(item)
                image_epoch = extract_epoch_from_segment(file_segment)
                image_datetime = datetime.fromtimestamp(image_epoch / 1000).strftime('%Y-%m-%d %H:%M:%S') if image_epoch else ""

                # Extract post metadata
                entity_urn = safe_get(item, ["metadata", "backendUrn"])
                header = safe_get(item, ["header", "text", "text"])

                # Extract reshared post (if any)
                resharedUpdate = safe_get(item, ["*resharedUpdate"])
                resharedUpdate_id = extract_entity_id(resharedUpdate)
                entity_id = extract_entity_id(entity_urn)

                # Extract additional actor info (header profile, company, etc.)
                header_attributes = safe_get(item, ["header", "text", "attributesV2"], [])
                header_profile = safe_get(header_attributes[0], ["detailData", "*profileFullName"]) if header_attributes else ""
                header_profile_id = header_profile.split("profile:")[1] if "profile:" in header_profile else ""

                commentary_attributes = safe_get(item, ["commentary", "text", "attributesV2"], [])
                company_id = safe_get(commentary_attributes[0], ["detailData", "*companyName"]) if commentary_attributes else None

                # Extract social detail links (used for linking shared posts)
                socialDetail = safe_get(item, ["*socialDetail"])
                matches = re.findall(r'urn:li:(?:activity|ugcPost):(\d+)\b|urn:li:groupPost:\d+-(\d+)\b', socialDetail)
                flattened_matches = [m[0] or m[1] for m in matches if m[0] or m[1]]
                sd_left = flattened_matches[0] if len(matches) >= 2 else None
                sd_right = flattened_matches[1] if len(matches) >= 2 else None

                # Build post dictionary
                post_data = {
                    "entity_id": str(entity_id) if entity_id else "",
                    "resharedUpdate_id": str(resharedUpdate_id) if resharedUpdate_id else "",
                    "header": header,
                    "post_text": safe_get(item, ["commentary", "text", "text"]),
                    "actor_description": safe_get(item, ["actor", "description", "text"]),
                    "actor_name": safe_get(item, ["actor", "name", "text"]),
                    "actor_profile": actor_profile_id,
                    "actor_backendUrn": safe_get(item, ["actor", "backendUrn"]),
                    "share_url": safe_get(item, ["socialContent", "shareUrl"]),
                    "file_segment": file_segment,
                    "image_epoch": image_epoch,
                    "image_datetime": image_datetime,
                    "socialDetail": socialDetail,
                    "sd_left": sd_left,
                    "sd_right": sd_right,
                    "header_profile_id": header_profile_id,
                    "company_id": company_id
                }

                # Add post to list
                posts.append(post_data)

            # === If this is Social Data (reactions, comments, shares) ===
            elif item.get("$type") == "com.linkedin.voyager.dash.feed.SocialActivityCounts":

                # Extract social data
                entity_urn = safe_get(item, ["entityUrn"])
                entity_id = extract_entity_id(entity_urn)

                # Raw counts
                numReactions = safe_get(item, ["numLikes"])
                numComments = safe_get(item, ["numComments"])
                numShares = safe_get(item, ["numShares"])

                # Detailed reaction types (like, interest, appreciation, etc.)
                reactionTypeCounts = safe_get(item, ["reactionTypeCounts"], [])
                reaction_map = {r.get("reactionType"): r.get("count", 0) for r in reactionTypeCounts}

                # Build social data dictionary
                sd = {
                    "entity_id": entity_id,
                    "numReactions": numReactions,
                    "numLikes": reaction_map.get("LIKE", 0),
                    "numInterests": reaction_map.get("INTEREST", 0),
                    "numAppreciates": reaction_map.get("APPRECIATION", 0),
                    "numEntertains": reaction_map.get("ENTERTAINMENT", 0),
                    "numEmpathys": reaction_map.get("EMPATHY", 0),
                    "numPraises": reaction_map.get("PRAISE", 0),
                    "numComments": numComments,
                    "numShares": numShares
                }

                # Add social data to list
                social_data.append(sd)

    # Done — return both lists
    return posts, social_data


# === Run Extraction ===

# Run the extraction function on combined HAR data
posts, social_data = extract_posts_and_social(combined_har_data)

# Print summary
print(f"✅ Extracted {len(posts)} posts and {len(social_data)} social records from ALL files")


---

### **What this step does**

We define a function:

`deduplicate_posts(posts)`

Its job is to:

- **Keep only ONE post** per unique `entity_id`
- If duplicates are found, prefer the version with **more complete data** (non-empty fields)

---

### **How it works**

1️⃣ We build a dictionary:

    best_posts = {}

- **Key** = `entity_id`
- **Value** = "best" version of the post for that ID

---

2️⃣ For each post:

- If it’s the **first time** seeing that `entity_id`, we store it
- If we already have a version:
    - We check each field
    - If the new version has **better data** (not `None` / not empty), we update the field

---

### **Why is this safe?**

- `entity_id` is a **LinkedIn post ID**. You only get one `entity_id` per post
- We know that any true duplicate posts will share this ID

---

### **Result**

After running:

    posts = deduplicate_posts(posts)

You will have a **clean list of unique posts** — one per LinkedIn post.

---

### **Summary printed**

Example output:

```
✅ Deduplicated to X unique posts
```
---

Next, we’ll start **building lookups** and enriching the posts with engagement data (likes, comments, shares).

---


In [ ]:
# === Deduplicate posts ===

def deduplicate_posts(posts):
    """
    Deduplicates posts based on 'entity_id'.

    Why?
    Sometimes multiple HAR files may capture the same post multiple times.
    We want ONE clean version of each post.

    Strategy:
    - Keep only ONE post per unique entity_id.
    - If we find duplicates, we prefer the post version that contains more complete data.

    Input:
    - posts: List of post dictionaries

    Returns:
    - List of deduplicated posts
    """

    best_posts = {}  # Dictionary to hold one "best" post per entity_id

    for post in posts:
        post_id = str(post.get("entity_id", ""))
        if not post_id:
            continue  # Skip posts with no ID (shouldn't happen, but safe)

        if post_id not in best_posts:
            # First time seeing this post — store it
            best_posts[post_id] = copy.deepcopy(post)
        else:
            # Already have a version of this post — check if this one has better data
            current_best = best_posts[post_id]

            # Go through each field
            for key, value in post.items():
                if key not in current_best:
                    current_best[key] = value
                elif current_best[key] in [None, "", [], {}] and value not in [None, "", [], {}]:
                    # If the current value is empty but this one has data — replace it
                    current_best[key] = value

    # Return the list of deduplicated posts
    return list(best_posts.values())


# === Run deduplication ===

posts = deduplicate_posts(posts)

# Summary print
print(f"✅ Deduplicated to {len(posts)} unique posts")


## 🔍 **Building Actor Lookup + Enriching Posts with Social Data**

---

### **What this step does:**

Now that we have extracted **post data** and **social engagement data** from the HAR files, we need to prepare the data for deeper analysis.

This section does two main things:

---

### 1️⃣ **Build an "Actor Lookup"**

- Many posts refer to *other users* (such as in **shares**, **comments**, or **reshared posts**).
- To avoid duplication and simplify enrichment, we build a simple **lookup dictionary**:
    - Keys: `actor_profile` (LinkedIn profile ID) or `actor_backendUrn`
    - Values: Actor details (name, description, profile)

This lets us easily **fill in missing info** for reshared posts later.

---

### 2️⃣ **Enrich Posts with Social Data + Classify Post Type**

We now loop through all extracted posts:

✅ **Enrich each post with social metrics** (likes, comments, shares, reactions).

✅ **Detect and handle reshared posts:**

- **Case 1:** *Share with no comment* → just a pure share → we label it `"Share No Comment"`.
    - If we don't already have the original post, we create a placeholder for it.
- **Case 2:** *Share with comment* → user adds their own commentary → we label it `"Share With Comment"`.
- **Case 3:** *Original post* → standalone post → we label it `"Original"`.

---

### **Why is this needed?**

👉 Many LinkedIn "shares" and "reshared posts" link **across posts**.

👉 The HAR data **does not always include the full original post** — so we must **track these links** and ensure we create a **clean, unified dataset** where:

- Post types are clear.
- Social metrics are attached.
- Links between shares and originals are preserved.
- Actor details are consistently populated.

---

### **Result:**

At the end of this step, we will have:

✅ All posts tagged as:
- `"Original"`
- `"Share With Comment"`
- `"Share No Comment"`

✅ All posts enriched with:
- Likes
- Comments
- Shares
- Reaction types

✅ Links between reshared posts and original posts clearly mapped.

✅ Cleaned fields — only the relevant data kept.

---

### **Summary:**

This is a **crucial cleaning step** before moving on to:  
➡️ **time-based analysis**  
➡️ **engagement trends**  
➡️ **network structure analysis** (who shares whose content).

---

🚀 Final print:
```
✅ Posts enriched and tagged.
```


In [ ]:
# === Build actor lookup ===

def build_actor_lookup(posts):
    """
    Creates a lookup dictionary of actors (post authors).

    Why?
    Some posts reference other users (such as in shares), and we want to
    easily look up their profile info (name, description, etc).

    The lookup uses:
    - actor_profile (profile ID string)
    - actor_backendUrn (LinkedIn backend URN)

    Returns:
    - Dictionary: { actor_key : { actor details } }
    """
    actor_lookup = {}

    for post in posts:
        # Use both profile ID and backend URN as possible keys
        for key in [post.get("actor_profile"), post.get("actor_backendUrn")]:
            if key:
                actor_lookup[key] = {
                    "actor_description": post.get("actor_description", ""),
                    "actor_name": post.get("actor_name", ""),
                    "actor_profile": post.get("actor_profile", ""),
                    "actor_backendUrn": post.get("actor_backendUrn", "")
                }

    return actor_lookup


# Build the lookup
actor_lookup = build_actor_lookup(posts)
print(f"✅ Built actor lookup with {len(actor_lookup)} actors")


# === Enrich posts with social data ===

# Step 1: Build social lookup
# This makes it fast to look up social metrics for each post
social_lookup = {str(sd["entity_id"]): sd for sd in social_data}

# Step 2: Track which entity_ids already exist (for deduplication when adding reshared posts)
existing_entity_ids = {post["entity_id"] for post in posts if "entity_id" in post}


# === Helper function to create a reshared post ===

def create_reshared_post(post):
    """
    Creates a minimal 'reshared' post record based on the original post.
    Useful when we detect a post was reshared but we have no original record yet.
    """
    return {
        "entity_id": post["resharedUpdate_id"],
        "resharedUpdate_id": "",
        "post_text": post["post_text"],
        "actor_description": post["actor_description"],
        "actor_name": post["actor_name"],
        "actor_profile": post["actor_profile"],
        "actor_backendUrn": post["actor_backendUrn"],
        "share_url": post["share_url"],
        "image_epoch": post["image_epoch"],
        "image_epoch_interpolated": np.nan,
        "interpolated_time": "",
        "numReactions": post.get("numReactions", ""),
        "numLikes": post.get("numLikes", ""),
        "numInterests": post.get("numInterests", ""),
        "numAppreciates": post.get("numAppreciates", ""),
        "numEntertains": post.get("numEntertains", ""),
        "numEmpathys": post.get("numEmpathys", ""),
        "numPraises": post.get("numPraises", ""),
        "numComments": post.get("numComments", ""),
        "numShares": post.get("numShares", ""),
        "type": "Original",  # Mark as original because it’s the original content
        "sd_left": "",
        "sd_right": ""
    }


# === Main loop: Enrich and tag posts ===

for post in posts:
    post_id = post.get("entity_id", "")

    # Add social metrics to this post if available
    social_record = social_lookup.get(post_id)
    if social_record:
        post.update({k: v for k, v in social_record.items() if k != "entity_id"})

    # === Handling reshared posts ===

    # Case 1: Reshare with NO COMMENT (just a pure share of another post)
    if post["sd_left"] != post["sd_right"] and post.get("type") is None:
        # If we don't already have this reshared post in our dataset, create it
        if post["sd_left"] not in existing_entity_ids:
            post["resharedUpdate_id"] = post["sd_left"]
            reshared_post = create_reshared_post(post)
            existing_entity_ids.add(reshared_post["entity_id"])
            posts.append(reshared_post)

        # Now update THIS post to mark it as "Share No Comment"
        ref_id = post.get("header_profile_id") or post.get("company_id")
        actor = actor_lookup.get(ref_id, {})

        post.update({
            "post_text": "",
            "actor_description": actor.get("actor_description", ""),
            "actor_profile": actor.get("actor_profile", ""),
            "actor_backendUrn": actor.get("actor_backendUrn", ""),
            "actor_name": actor.get("actor_name", ""),
            "share_url": "",
            "image_epoch": "",
            "image_epoch_interpolated": np.nan,
            "interpolated_time": "",
            "numReactions": "",
            "numLikes": "",
            "numInterests": "",
            "numAppreciates": "",
            "numEntertains": "",
            "numEmpathys": "",
            "numPraises": "",
            "numComments": "",
            "numShares": "",
            "type": "Share No Comment"
        })

    # Case 2: Reshare WITH COMMENT
    elif post["resharedUpdate_id"] and not post["header"] and post.get("type") is None:
        post["type"] = "Share With Comment"

    # Case 3: Original post
    elif (not post["resharedUpdate_id"]) and post.get("type") is None:
        post["type"] = "Original"

    # === Clean up unused fields ===
    for field in ["socialDetail", "sd_left", "sd_right", "header_profile_id", "company_id", "file_segment", "header"]:
        if field in post:
            del post[field]

print("✅ Posts enriched and tagged")


## 📅 **Interpolating Post Timestamps (image_epoch)**

### **Why is this step needed?**

When LinkedIn displays a post, it sometimes contains an embedded image (like a profile photo, attachment, or inline image). Conveniently, the LinkedIn image URLs include an *epoch timestamp* — which tells us roughly when the image was uploaded (and thus, when the post was made).

But:

- Not every post contains an image.
- If there's no image, we don't get an epoch timestamp from that post.
- However, LinkedIn post IDs (`entity_id`) are **sequential** → newer posts have higher IDs.

So we can **interpolate the missing timestamps** based on the `entity_id` order!

---

### 🕵️ **How this works**

1️⃣ If the post has an image with a valid timestamp → use it.  
2️⃣ If the post is missing a timestamp:
   - Find the nearest posts *before and after* it (based on entity_id) that DO have valid timestamps.
   - Interpolate a timestamp between them (linear interpolation).
3️⃣ If the post is at the very start (no "before" posts yet), we extrapolate **backward** using the first known slope.
4️⃣ If the post is at the very end (no "after" posts), we extrapolate **forward**.

---

### 🗂️ **What columns are added?**

✅ `image_epoch_interpolated` → A timestamp for every post (either real or interpolated).  
✅ `interpolated_time` → A human-readable datetime version (YYYY-MM-DD HH:mm:ss).

---

### 🔍 **Why this is safe**

- LinkedIn `entity_id` is monotonically increasing → post order is preserved.
- The gaps between posts are small enough that interpolated timestamps are very accurate (a few seconds/minutes of uncertainty at most).
- This allows us to **plot time trends**, **analyze posting patterns**, and **compare gaps between posts** even for those that lacked an image.

---

### 🛠️ **In this code cell:**

```python
df = pd.DataFrame(posts)
df = df.sort_values(by="entity_id", ascending=False)
df = interpolate_epochs(df)
```

---

At the end of this step, every post will have a timestamp — either directly from LinkedIn or inferred.


In [ ]:
# === Interpolate image_epoch ===

def interpolate_epochs(df):
    """
    Interpolates missing 'image_epoch' values for posts.

    Why?
    - Some LinkedIn posts do not contain an image (and thus no epoch timestamp from the image URL).
    - But 'entity_id' is a sequential ID (higher entity_id = newer post).
    - We can infer missing timestamps by interpolating between known points.

    What it does:
    - Fills in 'image_epoch_interpolated' for all posts.
    - Also adds a human-readable 'interpolated_time' column.

    Approach:
    - Forward interpolation between known image_epoch values.
    - Handles leading gaps (early records) with extrapolation.
    """
    known_points = []  # (entity_id, image_epoch) pairs where we know the timestamp
    early_records = [] # Posts at the start with no known earlier points
    slope = None       # Slope of entity_id → epoch (used for extrapolation)
    first_known_entity_id = None
    first_known_epoch = None

    # Ensure image_epoch is numeric
    df["image_epoch"] = pd.to_numeric(df["image_epoch"], errors="coerce")

    # Go through each post
    for idx, row in df.iterrows():
        entity_id_numeric = int(row["entity_id"])
        current_epoch = row["image_epoch"]

        if pd.notna(current_epoch):
            # We have a known epoch → record it
            known_points.append((entity_id_numeric, float(current_epoch)))
            df.at[idx, "image_epoch_interpolated"] = current_epoch

            # If we now have at least 2 known points, we can compute slope
            if len(known_points) >= 2:
                eid_before, epoch_before = known_points[-2]
                eid_after, epoch_after = known_points[-1]
                slope = (epoch_after - epoch_before) / (eid_after - eid_before)

                # Save first known point (for leading gaps)
                if first_known_entity_id is None:
                    first_known_entity_id = eid_before
                    first_known_epoch = epoch_before

                # Fill in early records (those that came BEFORE first known point)
                for early_idx, early_eid in early_records:
                    extrapolated_epoch = first_known_epoch - slope * (first_known_entity_id - early_eid)
                    df.at[early_idx, "image_epoch_interpolated"] = int(round(extrapolated_epoch))
                    df.at[early_idx, "interpolated_time"] = datetime.fromtimestamp(df.at[early_idx, "image_epoch_interpolated"] / 1000).strftime('%Y-%m-%d %H:%M:%S')

                early_records.clear()

        else:
            # This post is missing image_epoch — interpolate
            if len(known_points) >= 2:
                # Find known points before and after this post
                before = [pt for pt in known_points if pt[0] < entity_id_numeric]
                after = [pt for pt in known_points if pt[0] > entity_id_numeric]

                # Case 1: Both before and after points exist — true interpolation
                if before and after:
                    eid_before, epoch_before = max(before, key=lambda x: x[0])
                    eid_after, epoch_after = min(after, key=lambda x: x[0])
                    local_slope = (epoch_after - epoch_before) / (eid_after - eid_before)
                    interpolated_epoch = epoch_before + local_slope * (entity_id_numeric - eid_before)

                # Case 2: Only before points — extrapolate forward
                elif before:
                    if len(before) >= 2:
                        b1, b2 = before[-2:]
                        local_slope = (b2[1] - b1[1]) / (b2[0] - b1[0])
                    else:
                        local_slope = slope or 0

                    eid_before, epoch_before = max(before, key=lambda x: x[0])
                    interpolated_epoch = epoch_before + local_slope * (entity_id_numeric - eid_before)

                # Case 3: Only after points — extrapolate backward
                elif after:
                    if len(after) >= 2:
                        a1, a2 = after[:2]
                        local_slope = (a2[1] - a1[1]) / (a2[0] - a1[0])
                    else:
                        local_slope = slope or 0

                    eid_after, epoch_after = min(after, key=lambda x: x[0])
                    interpolated_epoch = epoch_after - local_slope * (eid_after - entity_id_numeric)

                else:
                    # No known points at all — can't interpolate
                    interpolated_epoch = 0

                df.at[idx, "image_epoch_interpolated"] = int(round(interpolated_epoch))

            else:
                # Not enough known points yet — store this for later extrapolation
                early_records.append((idx, entity_id_numeric))
                df.at[idx, "image_epoch_interpolated"] = np.nan

        # Add human-readable time column
        if pd.notna(df.at[idx, "image_epoch_interpolated"]):
            df.at[idx, "interpolated_time"] = datetime.fromtimestamp(df.at[idx, "image_epoch_interpolated"] / 1000).strftime('%Y-%m-%d %H:%M:%S')

    return df


# === Run interpolation ===

# Prepare dataframe
df = pd.DataFrame(posts)
df = df.sort_values(by="entity_id", ascending=False)

# Run interpolation
df = interpolate_epochs(df)
print("✅ Interpolation complete")

## 📤 **Output: Save Data to Excel**

In this step, we save our **cleaned and structured post data** to an Excel file.  
This gives us a permanent snapshot of all extracted post information — ready for:

✅ Manual exploration in Excel or Google Sheets  
✅ Sharing with others  
✅ Further analysis (in Python, R, etc.)  
✅ Feeding into the next **statistics section** of this notebook  

---

### ✨ **Why save now?**

- This export is a **full post-level dataset** — no summarisation yet.
- It captures:
  - Who posted what
  - When they posted it (real or interpolated time)
  - Engagement metrics
  - Links between shared posts
  - Post types (Original / Share With Comment / Share No Comment)

---

### 🗂️ **What's in the Excel file?**

| Column | Description |
|--------|-------------|
| `entity_id` | Unique ID for this post |
| `resharedUpdate_id` | If this post is a share, the ID of the original post |
| `post_text` | The full text of the post |
| `actor_description` | Bio/description of the author |
| `actor_name` | Name of the author |
| `actor_profile` | Profile ID string |
| `actor_backendUrn` | LinkedIn internal backend URN |
| `share_url` | Public URL to the post (if available) |
| `type` | Type of post (Original / Share With Comment / Share No Comment) |
| `numReactions` | Total number of reactions |
| `numLikes` | Count of likes |
| `numInterests` | Count of "Interest" reactions |
| `numAppreciates` | Count of "Appreciation" reactions |
| `numEntertains` | Count of "Entertainment" reactions |
| `numEmpathys` | Count of "Empathy" reactions |
| `numPraises` | Count of "Praise" reactions |
| `numComments` | Number of comments |
| `numShares` | Number of shares |
| `image_epoch` | Raw timestamp extracted from image URL (if available) |
| `image_epoch_interpolated` | Interpolated timestamp (filled for all posts) |
| `interpolated_time` | Human-readable post time (YYYY-MM-DD HH:MM:SS) |

---

### 📝 **Why this is useful**

- This file provides a **flat, consistent source of truth**.
- You can use this to:
  - Drive additional custom analysis.
  - Build charts and reports.
  - Cross-reference posts in LinkedIn.
  - Select variables to feed into your **stats & trends** section later in the notebook.

---

### 🚀 **How it works**

1️⃣ Ensures `entity_id` and `resharedUpdate_id` are stored as strings (for consistent export).  
2️⃣ Defines which **columns** to include in the Excel output.  
3️⃣ Uses `pd.ExcelWriter` to write the data to an `.xlsx` file using the XlsxWriter engine.  
4️⃣ Saves the file to:  
```
/content/drive/MyDrive/LinkedInData/linkedin_all_data.xlsx
```
This path/filename can be altered if you choose.

---

✅ When this runs successfully, you'll see:  
```
✅ Excel file created: /content/drive/MyDrive/LinkedInData/linkedin_all_data.xlsx
```

This file can now be used anywhere. It can even be used as the **input for the next analysis section** of this tutorial if you wish!

In [ ]:
# === Output: Save to Excel ===

# Why?
# We want an easy-to-use **structured output** — so we save it as an Excel file.
# This makes it simple to:
# - Explore in Excel / Google Sheets
# - Do further analysis
# - Share with others

# First: ensure these key fields are strings (for consistent export)
df["entity_id"] = df["entity_id"].astype(str)
df["resharedUpdate_id"] = df["resharedUpdate_id"].astype(str)

# Output path — adjust if you want a different location
output_path = "/content/drive/MyDrive/LinkedInData/linkedin_all_data.xlsx"

# === What we include in the output ===

# These columns give us:
# - Post identity
# - Who posted it
# - Content
# - Engagement stats
# - Timing (including interpolated time)
# - Post type (original / share)

show_columns = [
    "entity_id",                 # Unique post ID
    "resharedUpdate_id",         # If this is a share, what post was shared
    "post_text",                 # The text of the post
    "actor_description",         # Author bio/description
    "actor_name",                # Author name
    "actor_profile",             # Profile ID
    "actor_backendUrn",          # LinkedIn internal ID
    "share_url",                 # Public URL to this post
    "type",                      # Original, Share with comment, Share no comment
    "numReactions",              # Total reactions
    "numLikes",                  # Like count
    "numInterests",              # Interest reactions
    "numAppreciates",            # Appreciation reactions
    "numEntertains",             # Entertainment reactions
    "numEmpathys",               # Empathy reactions
    "numPraises",                # Praise reactions
    "numComments",               # Number of comments
    "numShares",                 # Number of shares
    "image_epoch",               # Raw timestamp (from image URL) — where available
    "image_epoch_interpolated",  # Interpolated timestamp (for all posts)
    "interpolated_time"          # Human-readable time (yyyy-mm-dd HH:MM:SS)
]

# === Save the file ===

# We use ExcelWriter with XlsxWriter engine (produces a modern .xlsx)
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Post_Data', index=False, columns=show_columns)

print("✅ Excel file created:", output_path)

## 📈 **Final Step: Generate and Export Post Statistics**

---

Now that we have a full, structured dataset of posts, we can generate some **summary statistics** — this is where the insights start to emerge!

---

### 🚀 **What this section does**

1️⃣ **Loads the cleaned Excel data** (optional if you've already run the notebook end-to-end).  
2️⃣ **Parses timestamps** — so we can analyze posts over time.  
3️⃣ **Filters for a single author** and only valid post types:  
   - `Original`  
   - `Share With Comment`  
   - `Share No Comment`  

4️⃣ **Calculates core metrics**:  
   - Posting date, week, year  
   - Word count  
   - Character count  
   - Emoji usage  
   - Hashtag usage  
   - Time gaps between posts (for burst analysis)  

5️⃣ **Creates the following summary tables**:
- **Daily Stats** → Posts & average word count per day
- **Weekly Stats** → Posts & average word count per week
- **Day of Week Stats** → Posting patterns by day of the week
- **Overall Metrics** → Totals and averages, split by post type

6️⃣ **Exports all of this to an Excel file** (`linkedin_combined_stats.xlsx`) — with multiple sheets:
- `Daily_Stats`
- `Weekly_Stats`
- `DayOfWeek_Stats`
- `Overall_Stats`
- `Enhanced_Post_Data` → Full post-level data with new metrics

---

### 🔍 **Why this is useful**

These stats allow you to explore questions like:

- **How frequently does this person post?**
- **Do they post in bursts, or consistently?**
- **How long are their posts on average?**
- **What days of the week are they most active?**
- **Are they producing "original" content or mostly shares?**
- **Has their activity changed over time?**

---

### ✏️ **Notes & Customization**

- You must replace the target **`actor_profile`** with any profile ID you want to analyze.

  This can be found in the Excel file output in the previous step.

  (This allows re-running the same analysis for multiple creators — just swap the ID.)
  ```python
filtered_df = df.loc[
    ((df["type"] == "Original") |
     (df["type"] == "Share With Comment") |
     (df["type"] == "Share No Comment")) &
    (df["actor_profile"] == "ACoAABDoFkIBu5s4sIdD-WTas39dsfSNq-XYDF")  # <-- Replace with your author!
].copy()
  ```

- You can also easily add more metrics here:
  - Average time gap between posts  
  - Post sentiment (using an LLM!)  
  - Engagement per post type  
  - And more...

---

### 📂 **Output Files**

The final Excel file will be saved to:

```python
output_path = "/content/drive/MyDrive/LinkedInData/linkedin_combined_stats.xlsx"
```

This can be changed if you wish.

---

### 🎁 **Why this design?**

- **Separation of concerns**:
  - The earlier "all_data" export is a **raw snapshot** of posts.
  - This step creates **derived stats** based on a target author and post type.

- **Flexibility**:
  - You can experiment and iterate on metrics without having to re-extract all HAR data.

- **Reproducibility**:
  - You can run this for any LinkedIn user whose posts you've captured.

---

✅ After this step, you'll have a complete dataset, ready for:

📊 Visualisation  
🧠 Interpretation  
🤖 LLM analysis (e.g. "Summarize this author's tone")  
📈 Comparative studies across creators  

In [ ]:
# === Load and Prepare Data for Analysis ===

# OPTIONAL: Load from Excel file created in previous step
"""
input_path = "/content/drive/MyDrive/LinkedInData/linkedin_all_data.xlsx"
df = pd.read_excel(input_path, sheet_name='Post_Data')

print(f"✅ Loaded {len(df)} posts from Excel")
"""

# === Parse datetime ===
df['image_datetime'] = pd.to_datetime(df['interpolated_time'], errors='coerce')

# Drop rows where datetime is missing
df = df.dropna(subset=['image_datetime'])

# === Filter: Target Author + Valid Post Types ===

# Why?
# - We want to focus on ONE author (replace with your target actor_profile)
# - Only posts of type Original / Share With Comment / Share No Comment

filtered_df = df.loc[
    ((df["type"] == "Original") |
     (df["type"] == "Share With Comment") |
     (df["type"] == "Share No Comment")) &
    (df["actor_profile"] == "ACoAABDoFkIBu5s4sIdD-WTas39dsfSNq-XYDF")  # <-- Replace with your author!
].copy()

# === Core Metrics ===

# Extract useful time dimensions
filtered_df['date'] = filtered_df['image_datetime'].dt.date
filtered_df['week'] = filtered_df['image_datetime'].dt.isocalendar().week
filtered_df['year'] = filtered_df['image_datetime'].dt.year

# Word count
filtered_df['word_count'] = filtered_df['post_text'].fillna("").apply(lambda x: len(str(x).split()))

# === Additional Metrics ===

filtered_df['day_of_week'] = filtered_df['image_datetime'].dt.day_name()
filtered_df['char_count'] = filtered_df['post_text'].fillna("").apply(len)
filtered_df['emoji_count'] = filtered_df['post_text'].fillna("").apply(lambda x: len(re.findall(r'[^\w\s,]', x)))
filtered_df['hashtag_count'] = filtered_df['post_text'].fillna("").apply(lambda x: len(re.findall(r'#\w+', x)))

# Calculate time gaps between posts
filtered_df = filtered_df.sort_values('image_datetime')
filtered_df['post_gap_seconds'] = filtered_df['image_datetime'].diff().dt.total_seconds().fillna(0)
filtered_df['post_gap_minutes'] = filtered_df['post_gap_seconds'] / 60
filtered_df['post_gap_hours'] = filtered_df['post_gap_seconds'] / 3600

# === Daily Stats ===

posts_pivot = filtered_df.pivot_table(
    index='date',
    columns='type',
    values='entity_id',
    aggfunc='count',
    fill_value=0
).reset_index()

words_pivot = filtered_df.pivot_table(
    index='date',
    columns='type',
    values='word_count',
    aggfunc='mean',
    fill_value=0
).reset_index()

daily_stats = posts_pivot.merge(words_pivot, on='date', suffixes=(' Count', ' Avg Words'))

# === Weekly Stats ===

weekly_posts_pivot = filtered_df.pivot_table(
    index=['year', 'week'],
    columns='type',
    values='entity_id',
    aggfunc='count',
    fill_value=0
).reset_index()

weekly_words_pivot = filtered_df.pivot_table(
    index=['year', 'week'],
    columns='type',
    values='word_count',
    aggfunc='mean',
    fill_value=0
).reset_index()

weekly_stats = weekly_posts_pivot.merge(weekly_words_pivot, on=['year', 'week'], suffixes=(' Count', ' Avg Words'))

# === Day of Week Stats ===

day_of_week_posts_pivot = filtered_df.pivot_table(
    index='day_of_week',
    columns='type',
    values='entity_id',
    aggfunc='count',
    fill_value=0
).reset_index()

day_of_week_words_pivot = filtered_df.pivot_table(
    index='day_of_week',
    columns='type',
    values='word_count',
    aggfunc='mean',
    fill_value=0
).reset_index()

day_of_week_stats = day_of_week_posts_pivot.merge(day_of_week_words_pivot, on='day_of_week', suffixes=(' Count', ' Avg Words'))

# === Overall Metrics (by Type) ===

overall_by_type = filtered_df.groupby('type').agg(
    total_posts=('post_text', 'count'),
    unique_post_days=('date', 'nunique'),
    average_posts_per_day=('date', lambda x: x.value_counts().mean()),
    average_word_count=('word_count', 'mean'),
    max_word_count=('word_count', 'max'),
    min_word_count=('word_count', 'min'),
    most_active_day=('date', lambda x: x.value_counts().idxmax()),
    most_active_day_count=('date', lambda x: x.value_counts().max())
).reset_index()

# === Export All Stats to Excel ===

output_path = "/content/drive/MyDrive/LinkedInData/linkedin_combined_stats.xlsx"

with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    # Save daily stats
    daily_stats.to_excel(writer, sheet_name='Daily_Stats', index=False)

    # Save weekly stats
    weekly_stats.to_excel(writer, sheet_name='Weekly_Stats', index=False)

    # Save day-of-week stats
    day_of_week_stats.to_excel(writer, sheet_name='DayOfWeek_Stats', index=False)

    # Save overall metrics by post type
    overall_by_type.to_excel(writer, sheet_name='Overall_Stats', index=False)

    # Save FULL enhanced post-level data
    enhanced_columns = [
        "entity_id",
        "resharedUpdate_id",
        "post_text",
        "actor_description",
        "actor_name",
        "actor_profile",
        "actor_backendUrn",
        "share_url",
        "type",
        "numReactions",
        "numLikes",
        "numInterests",
        "numAppreciates",
        "numEntertains",
        "numEmpathys",
        "numPraises",
        "numComments",
        "numShares",
        "image_epoch",
        "image_epoch_interpolated",
        "interpolated_time",
        "date",
        "week",
        "year",
        "word_count",
        "day_of_week",
        "char_count",
        "emoji_count",
        "hashtag_count",
        "post_gap_seconds",
        "post_gap_minutes",
        "post_gap_hours"
    ]

    filtered_df.to_excel(writer, sheet_name='Enhanced_Post_Data', index=False, columns=enhanced_columns)

print("✅ Excel file created:", output_path)